# Perceptual Losses for Real-Time Style Transfer and Super-Resolution

## Abstract

我们考虑图像变换的问题，其中输入图像被转换成输出图像。针对这些问题，最近的方法通常使用输出图像与真实图像之间的每像素损失来训练前馈卷积神经网络。最近工作表明，通过定义和优化基于从预训练网络提取的高级特征的感知损失函数，可以生成高质量的图像。我们结合这两种方法的好处，并提出使用感知损失函数训练用于图像转换任务的前馈网络。我们展示了图像风格转换的结果，其中前馈网络被训练以解决由Gatys等人提出的实时优化问题。与基于优化的方法相比，我们的网络给出了类似的定性结果，但速度提高了三个数量级。我们还尝试使用单图像超分辨率，其中用感知损失替换每像素损失给出视觉上令人满意的结果。

## Introduction 

许多经典问题可以被定义为图像转换任务，系统接收一些输入图像并将其转换为输出图像。来自图像处理的示例包括去噪，超分辨率和彩色化，其中输入是劣化图像（噪声，低分辨率或灰度），并且输出是高质量彩色图像。来自计算机视觉的示例包括语义分割和深度估计，其中输入是彩色图像并且输出图像编码关于场景的语义或几何信息。  

解决图像转换任务的一种方法是以监督的方式训练前馈卷积神经网络，使用每像素损失函数来测量输出和真实图像之间的差异。这种方法已经被Dong等人用于超分辨率[1]，Cheng等人着色[2]，Long等人[3]以及Eigen等人对深度和表面法线预测[4,5]。这种方法在测试时是高效的，只需要通过训练的网络的前向传递。  

然而，这些方法所使用的每像素损失并不能捕获输出图像与真实图像之间的感知差异。例如，考虑两个相同的图像相互偏移一个像素; 尽管他们的感知相似性，他们测量到的每像素损失将是非常不同的。  

同时，最近的工作表明，高质量的图像可以使用感知损失函数而不是像素之间的差异，从预训练卷积神经网络中提取的高级图像特征表示之间的差异来生成。 图像是通过最小化损失函数生成的。Mahendran等[6]将这一策略应用于Simonyan等[7]和Yosinski等[8]的特征可视化，以及Gatys等[9,10]的纹理合成和样式转换。这些方法产生高质量的图像，但由于推理需要解决优化问题，所以速度较慢。  

在本文中，我们结合这两种方法的好处。 我们训练用于图像转换任务的前馈变换网络，而不是仅依赖于低级像素信息来使用每像素丢失函数，而是使用依赖于预训损失网络的高级特征的感知丢失函数来训练我们的网络。在训练期间，感知损失比每像素损失更加稳健地测量图像相似性，并且在测试时间，转换网络实时运行。  

我们试验了两个任务：样式转换和单图超分辨率。两者本质上都是不适定的; 对于样式转换，没有单个正确的输出，而对于超分辨率，有许多高分辨率图像可能产生相同的低分辨率输入。任何一个任务的成功都需要关于输入图像的语义推理。对于样式转换，输出必须在语义上与输入类似，尽管颜色和纹理发生剧烈变化; 对于超分辨率，必须从视觉模糊的低分辨率输入中推断出细节。原则上，为任一任务训练的高容量神经网络可以隐含地学习推理相关的语义; 然而在实践中我们不需要从零开始学习：使用感知丢失函数可以将语义知识从丢失网络转移到转换网络。  

对于样式转换，我们的前馈网络被训练来解决[10]中的优化问题; 我们的结果在定性和定量上都与目标函数值相似[10]，但是产生的速度要快三个数量级。 对于超分辨率，我们表明，用感知损失替换每像素损失给予×4和×8超分辨率的视觉上令人满意的结果。  

##  Related Work 

前馈图像转换  

近年来，通过训练具有每像素损失函数的深度卷积神经网络，已经解决了各种各样的前馈图像变换任务。

语义分割方法通过在输入图像上以全卷积方式运行网络来产生密集的场景标签，训练具有逐像素分类损失。通过将CRF推断作为与网络其余部分共同训练的递归层，超越了每像素点的损失。我们的转换网络的架构受到[3]和[14]的启发，它们使用网内下采样来减少特征映射的空间范围，随后通过网内上采样来产生最终的输出图像。  

最近的深度方法和表面法线估计是类似的，他们使用一个前馈卷积网络，使用 perpixel回归训练，将彩色输入图像转换成几何意义上的输出图像或分类损失。有些方法通过惩罚图像梯度或使用CRF损失层来增强每像素点的损失，从而强化输出图像中的局部一致性。在[2]前馈模型是训练使用每像素损失将灰度图像转换为彩色。  

感知优化  

最近的一些论文使用优化来生成图像，作为目标函数的感知损失依赖于卷积网络提取的高级特征。为了理解在训练过的网络中编码的功能，可以生成图像以最大化类别预测分数[7,8]或单个特征[8]。类似的优化技术也可以用来生成高可信度的假图像。  

Mahendran和Vedaldi [6]通过最小化特征重构损失来反演卷积网络的特征，以便理解不同网络层保留的图像信息;之前已经使用类似的方法来反演局部二进制描述符[20]和HOG特征[21]。  

Dosovitskiy和Brox [22]的工作与我们的工作特别相关，因为他们训练一个前馈神经网络来反演卷积特征，快速逼近解决[6]提出的优化问题。然而，他们的前馈网络训练与每像素重建损失，而我们的网络直接优化特征重建损失[6]。  

风格转移  

Gatys等[10]进行艺术风格的转移，将一幅图像的内容与另一幅图像的内容相结合，通过联合最小化[6]的特征重构损失和基于从预训练卷积网络提取的特征的样式重建损失;之前已经有一种类似的方法用于纹理合成[9]。他们的方法产生了高质量的结果，但是由于优化问题的每一步需要向前和向后通过预训练网络，所以在计算上是昂贵的。为了克服这个计算负担，我们训练一个前馈网络来快速逼近他们的优化问题的解决方案。  

图像超分辨率  

图像超分辨率是一个经典的问题，其中已经开发了各种各样的技术。Yang等[23]对广泛采用卷积神经网络之前的主流技术进行了详尽的评估。他们将超分辨率技术分为基于预测的方法（双线性，双三次，Lanczos [24]），基于边缘的方法[25,26]，统计方法[27,28,29]，基于补丁的方法和稀疏字典方法[37,38]。最近文献[1]利用三层卷积神经网络对每像素欧几里得损失进行训练，在单幅图像超分辨率上取得了优异的性能。其他最近的最先进的方法包括[39,40,41]。

##  Method 

如图2所示，我们的系统由两部分组成：一个图像变换网络$f_W$和一个用来定义几个损失函数的损耗网络$\phi$。 图像变换网络是一个深度残差卷积神经网络，由权重 W 参数化;它通过映射 $\overline y  = f_W(x)$ 将输入图像 x 转换成输出图像 y 。每个损失函数计算测量输出图像 y 与目标图像 $y_i$ 之间的差的标量值$l_i(\overline y,y_i)$。图像变换网络使用随机梯度下降来训练以最小化损失函数的加权组合：  

公式  

为了解决每像素损失的缺点，并允许我们的损失函数更好地衡量图像之间的感知和语义差异，我们从最近通过优化生成图像的工作中汲取灵感。这些方法的关键点是预处理图像分类的卷积神经网络已经学会了编码我们想要在我们的损失函数中测量的感知和语义信息。因此，为了定义我们的损失函数，我们利用一个已经被训练为分类的网络$\phi$作为一个固定的损失网络。然后我们使用也是深度卷积网络的损失函数对深度卷积变换网络进行训练。  

损失网络$\phi$用于定义测量图像之间的内容和风格的差异的特征重构损失$l_{feat}^{\phi}$和风格重建损失$l_{style}^{\phi}$。对于每个输入图像 x，我们都有一个内容目标$y_c$和一个样式目标$y_s$。对于样式转换，内容目标$y_c$是输入图像 x，输出图像 $\overline y$ 应该将$x = y_c$的内容与$y_s$的样式相结合; 我们为每个样式目标训练一个网络。对于单幅图像超分辨率，输入图像 x 是低分辨率输入，内容目标$y_c$是真实高分辨率图像，不使用样式重构损失;每个超分辨率因素训练一个网络。  

### Image Transformation Networks  

我们的图像转换网络大致遵循Radford等[42]提出的架构指导原则。我们不使用任何 pooling 层，而是使用 strided 和 fractionally strided 的卷积来进行网内下采样和上采样。我们的网络体由五个残余块组成[43]，使用[44]的体系结构。除了输出层之外，所有非残留卷积层之后都是空间批量归一化[45]和ReLU非线性，而输出层使用缩放的 tanh 来确保输出图像的像素在[0,255]范围内。除了使用9×9内核的第一层和最后一层以外，所有卷积层都使用3×3内核。我们所有网络的确切架构可以在补充材料中找到。  

输入和输出： 对于风格转移，输入和输出都是形状为3×256×256的彩色图像。对于具有上采样因子 f 的超分辨率，输出是形状为3×288×288的高分辨率图像块，输入是形状为3×288 / f×288 / f的低分辨率片。由于图像变换网络是完全卷积的，在测试时它们可以应用于任何分辨率的图像。  

下采样和上采样： 对于上采样因子为f的超分辨率，我们使用几个残余块，其次是步长为1/2的 $\log_2 f$ 卷积层。 这与[1]使用双三次插值在将低分辨率输入传递到网络之前上采样不同。上行采样函数可以与网络的其他部分共同学习，而不是依靠固定的上采样函数。  

对于样式传输，我们的网络使用两个步长2的卷积来对输入进行下采样，接着是几个残余块，然后使用两个卷积层，步长1/2来上采样。尽管输入和输出具有相同的大小，但对于下采样然后上采样的网络来说有几个好处。  

首先是计算，在一个简单的实现中，在尺寸为C×H×W的输入上使用 C 滤波器的3×3卷积需要$9HWC^2$的乘加，这与在形状 DC 的输入上具有 DC 滤波器的3×3卷积的成本相同 DC × H / D × W / D。下采样后，我们可以使用一个更大的网络相同的计算成本。  

第二个好处是有效的接受字段大小。高质量的样式转换需要以一致的方式更改大部分图像; 因此输出中的每个像素在输入中具有大的有效感受场是有利的。 在没有下采样的情况下，每个额外的3×3卷积层将有效感受野大小增加2倍。在以 D 倍下采样之后，每个3×3卷积反而通过 2D 增加有效感受野大小，给出具有相同数量的更大有效感受野图层。  

残差连接： He等[43]使用残差连接来训练非常深的网络进行图像分类。他们认为，残差连接使网络很容易学习识别功能; 这对于图像变换网络来说是一个吸引人的特性，因为在大多数情况下，输出图像应该与输入图像共享结构。因此，我们网络的主体由多个残余块组成，每个块包含两个3×3卷积层。我们使用补充材料中显示的[44]的残余块设计。  

###  Perceptual Loss Functions  

我们定义了两个感知损失函数，用于度量图像之间的高级感知和语义差异。用了一个用来进行图像分类的网络$\phi$作为损失网络，这意味着这些知觉损失函数本身是深度卷积神经网络。在我们所有的实验中，$\phi$是在ImageNet数据集上预训练的16层VGG网络[47]。  

特征重建损失：不是鼓励输出图像的像素 $\overline y  = f_W(x)$与目标图像 y 的像素完全匹配，而是鼓励它们具有与由损失网络$\phi$计算的相似的特征表示。令$\phi_j(x)$为处理图像 x 时网络的第 j 层的激活; 如果 j 是卷积层，则$\phi_j(x)$将是形状 $C_j$ × $H_j$ × $W_j$的特征映射。特征重构损失是特征表示之间的（平方，归一化）欧几里得距离：  

公式

如[6]中所示和图3中所示，寻找使早期层的特征重建损失最小化的图像 y 往往产生与 y 不可区分的图像。当我们从更高层重建时，图像内容和整体空间结构被保留，但颜色，纹理和确切的形状不是。使用特征重构损失来训练我们的图像变换网络，鼓励输出图像y在感觉上与目标图像y相似，但是不强制它们精确匹配。  

风格重建损失: 当特征重建损失与目标 y 的内容偏离时，特征重构损失会对输出图像 y 进行惩罚。为了达到这个效果，Gatys等[9,10]提出了以下样式重建的损失，我们也希望惩罚不同的风格：颜色，纹理，常见模式等。  

如上所述，令$\phi_j(x)$为输入 x 的网络$\phi$的第 j 层的激活，其是形状$C_j$ × $H_j$ × $W_j$的特征映射。将Gram矩阵$G(\phi_j(x))$定义为其元素由下式给出的$C_j$ × $C_j$矩阵  

公式  

如果我们将$\phi_j(x)$解释为给出 $H_j$ × $W_j$ 网格上每个点的 $C_j$ 维特征，那么$G(\phi_j(x))$与$C_j$维特征的非中心协方差成正比，将每个网格位置视为独立样品。因此，它捕获有关哪些功能一起激活的信息。通过将$\phi_j(x)$整形成形状为$C_j$ × $H_j$ × $W_j$的矩阵$\psi$，可以有效地计算出格拉姆矩阵。

样式重建损失是输出图像和目标图像的格拉姆矩阵之间差异的平方Frobenius范数：  

公式  

即使当 $\overline y$ 和 y 具有不同的尺寸时，样式重建损失也是明确的，因为它们的格式矩阵将具有相同的形状。  

正如文献[10]和图5所示，生成一个图像y，尽量减少样式重建损失，保留目标图像的风格特征，但不保留其空间结构。从更高层重建从目标图像转移更大规模的结构。  

为了从一组层 J 而不是单层  执行样式重构，我们定义 $L_{style}^{\phi,J}(\overline y,y)$ 为每层j∈J的损失总和。  

###  Simple Loss Functions  

除了上面定义的感知损失之外，我们还定义了两个简单的损失函数，它们只依赖于低级像素信息。  

像素损失。像素损失是输出图像y与目标y之间的（归一化的）欧几里德距离。如果两者都具有C×H×W的形状，则像素损失被定义为$l_{pixel}(\overline y,y) = \frac{{||\overline y - y||}_2^2}{CHW}$。 这只有当我们有一个真实目标并且与网络预期匹配时才能使用。  

全变分正则化。为了鼓励输出图像 y 的空间平滑，我们遵循特征反演[6,20]和超分辨[48,49]的先前工作，并利用全变分正则化函数$l_{TV}(\overline y)$。  

## Conclusion 

在本文中，我们通过训练具有感知损失函数的前馈变换网络，将前馈图像变换任务和基于优化的图像生成方法的优点结合起来。 我们已经将这种方法应用于样式转换，我们实现了可比较的性能，并且与现有方法相比大大提高了速度，并且在单一图像超分辨率的情况下，我们表明具有感知损失的训练允许模型更好地重构细节和边缘。  

在未来的工作中，我们希望探索使用感知损失函数来进行其他图像转换任务，如着色和语义分割。我们还计划调查使用不同的损失网络，看看例如在不同的任务或数据集上训练的损失网络是否可以传递具有不同类型的语义知识的图像转换网络。